In [ ]:
# 0. Load keras package needed
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.layers import Activation
from keras.layers import Dropout
from IPython.display import SVG
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
# Fix random seed
np.random.seed(3)

# Set hyper parameter and variables
# you must change that
batch_size = 10
num_epochs = 10
steps_epoch = 100
val_steps = 10
img_col = 224
img_row = 224
img_channel = 3
num_classes = 10 

# 1. Creating Datasets
train_datagen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15, # Rotation original image 15 degree
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.05, 
                                  zoom_range=[1.0, 1.2],  # Zome randomly in 1~1.2 ratio
                                  horizontal_flip=True, # True/False. On/Off
                                  vertical_flip=False,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        'dataset/train', # data drectory
        target_size=(img_col, img_row), # load image size
        batch_size=batch_size, 
        class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(img_col, img_row),
        batch_size=batch_size,
        class_mode='categorical')

# 2. Buliding a Model
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_col, img_row, img_channel))
model.trainable = False
x = model.output
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(input = model.input, output = predictions)

# 3. Setting up the Model Learning Process & 4. Training the Model
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=steps_epoch,
        epochs=num_epochs,
        validation_data=test_generator,
        validation_steps=val_steps)

# Model Architecture Visualization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# 5. Confirm the Learning Process
%matplotlib inline 
import matplotlib.pyplot as plt
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(hist.history['loss'], 'y', label='train loss')  
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()

# 6. Evaluating the Model
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
print("-- Predict --")
output = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)
print(test_generator.filenames)

In [ ]:
# 7. Using the Model
from keras.models import load_model

model_final.save('Classification_model_Inception.h5')